# Experiment: Gradient Loss for Improved Gamma Pass Rate

**Date:** 2026-01-20  
**Experiment ID:** `grad_loss_0.1`  
**Status:** Complete  
**Git Commit:** `5d111a0`  

---

## 1. Overview

### 1.1 Objective

Test whether adding a 3D Sobel gradient loss to the baseline U-Net improves the Gamma pass rate (3%/3mm) while maintaining dose accuracy (MAE).

**Hypothesis:** Adding gradient loss will improve edge preservation in dose predictions, leading to better Gamma pass rates without sacrificing overall dose accuracy.

### 1.2 Key Results

| Metric | Baseline | Gradient Loss | Change |
|--------|----------|---------------|--------|
| **Val MAE** | 3.73 Gy | **3.67 Gy** | -1.6% |
| **Test MAE** | 1.43 Gy | **1.44 Gy** | Same |
| **Gamma (3%/3mm)** | 14.2% | **27.9%** | **+96%** |

### 1.3 Conclusion

**SUCCESS:** The 3D Sobel gradient loss nearly doubled the Gamma pass rate (14.2% → 27.9%) while maintaining equivalent MAE. This validates that edge/gradient preservation is important for clinically acceptable dose distributions. Recommend proceeding with Phase B (gradient + VGG combined loss).

---

## 2. Reproducibility Information

In [ ]:
# Reproducibility information for this experiment
REPRODUCIBILITY_INFO = {
    'git_commit': '5d111a0',
    'git_message': 'docs: Add Claude Code WSL→Windows passthrough instructions',
    'python_version': '3.12.12',
    'pytorch_version': '2.6.0+cu124',
    'cuda_version': '12.4',
    'gpu': 'NVIDIA GeForce RTX 3090 (24 GB)',
    'random_seed': 42,
    'experiment_date': '2026-01-20',
    'training_script': 'scripts/train_baseline_unet.py',
    'figure_script': 'scripts/generate_grad_loss_figures.py',
}

print('Reproducibility Information:')
for k, v in REPRODUCIBILITY_INFO.items():
    print(f'  {k}: {v}')

### Command to Reproduce

```bash
# Checkout correct commit
git checkout 5d111a0

# Activate environment (Windows)
call C:\pinokio\bin\miniconda\Scripts\activate.bat vmat-win
cd C:\Users\Bill\vmat-diffusion-project

# Run training
python scripts\train_baseline_unet.py \
    --exp_name grad_loss_0.1 \
    --data_dir I:\processed_npz \
    --use_gradient_loss \
    --gradient_loss_weight 0.1 \
    --epochs 100

# Run inference on test set
python scripts\inference_baseline_unet.py \
    --checkpoint runs\grad_loss_0.1\checkpoints\best-epoch=012-val\mae_gy=3.670.ckpt \
    --input I:\processed_npz\case_0007.npz \
    --output_dir predictions\grad_loss_0.1_test

# Generate figures
python scripts\generate_grad_loss_figures.py
```

---

## 3. Dataset

In [ ]:
DATASET_INFO = {
    'total_cases': 23,
    'train_cases': 19,
    'val_cases': 2,
    'test_cases': 2,
    'test_case_ids': ['case_0007', 'case_0021'],
    'preprocessing_version': 'v2.2.0',
    'data_location': 'I:\\processed_npz',
    'split_seed': 42,
}

print('Dataset Information:')
for k, v in DATASET_INFO.items():
    print(f'  {k}: {v}')

**Dataset Details:**
- **Disease site:** Prostate cancer with SIB (Simultaneous Integrated Boost)
- **Prescription:** PTV70: 70 Gy, PTV56: 56 Gy in 28 fractions
- **Input channels (9):** CT, 8 SDF channels (structures)
- **Constraint conditioning (13):** DVH constraints via FiLM
- **Output:** 3D dose distribution (normalized 0-1, scaled by 70 Gy)

---

## 4. Model / Method

In [ ]:
MODEL_CONFIG = {
    'architecture': 'BaselineUNet3D (Direct Regression)',
    'parameters': '23,732,801 (23.73M)',
    'in_channels': 9,
    'out_channels': 1,
    'base_channels': 48,
    'constraint_dim': 13,
    'conditioning': 'FiLM (Feature-wise Linear Modulation)',
}

print('Model Configuration:')
for k, v in MODEL_CONFIG.items():
    print(f'  {k}: {v}')

### 4.1 Gradient Loss (GradientLoss3D)

The key addition in this experiment is the **3D Sobel gradient loss**, which encourages the model to preserve sharp dose gradients:

```python
class GradientLoss3D(nn.Module):
    """3D Sobel gradient loss for edge preservation."""
    
    def forward(self, pred, target):
        # Compute 3D Sobel gradients for pred and target
        pred_grad = sobel_3d(pred)   # [B, 3, D, H, W]
        target_grad = sobel_3d(target)
        
        # L1 loss on gradient magnitude
        return F.l1_loss(pred_grad, target_grad)
```

**Rationale:** Clinical dose distributions have sharp gradients at PTV boundaries. Standard MSE loss tends to produce "blurred" predictions. Gradient loss penalizes mismatched edges.

---

## 5. Training Configuration

In [ ]:
TRAINING_CONFIG = {
    'max_epochs': 100,
    'actual_epochs': 62,
    'early_stopping_patience': 50,
    'batch_size': 2,
    'patch_size': 128,
    'patches_per_volume': 4,
    'learning_rate': 1e-4,
    'optimizer': 'AdamW',
    'weight_decay': 0.01,
    'loss_function': 'MSE + Gradient Loss + Negative Penalty',
    'mse_weight': 1.0,
    'gradient_loss_weight': 0.1,
    'negative_penalty_weight': 0.1,
    'use_vgg_loss': False,
    'training_time': '1.85 hours',
}

print('Training Configuration:')
for k, v in TRAINING_CONFIG.items():
    print(f'  {k}: {v}')

---

## 6. Results

### 6.1 Training Curves

![Training Curves](../runs/grad_loss_0.1/figures/fig1_training_curves.png)

**Figure 1:** (A) Training and validation loss over epochs. (B) Validation MAE with best epoch marked. The model achieved best validation MAE of 3.67 Gy at epoch 12, slightly better than the baseline (3.73 Gy).

### 6.2 Model Comparison

![Model Comparison](../runs/grad_loss_0.1/figures/fig2_model_comparison.png)

**Figure 2:** Comparison of baseline U-Net vs gradient loss model. (A) Validation MAE. (B) Test MAE. (C) Gamma pass rate (3%/3mm). The gradient loss model nearly doubled the Gamma pass rate (+13.7%) while maintaining equivalent MAE.

### 6.3 Dose Visualization

![Dose Slices](../runs/grad_loss_0.1/figures/fig3_dose_slices.png)

**Figure 3:** Dose distribution for case_0007 (central axial slice). (A) Ground truth. (B) Prediction. (C) Difference map. The model captures the overall dose shape well, with small differences primarily at structure boundaries.

### 6.4 Loss Components

![Loss Components](../runs/grad_loss_0.1/figures/fig4_loss_components.png)

**Figure 4:** Breakdown of training loss components. The MSE loss dominates, with gradient loss providing a smaller but consistent contribution throughout training.

### 6.5 Quantitative Results

In [ ]:
RESULTS = {
    'best_val_mae_gy': 3.67,
    'best_epoch': 12,
    'test_mae_case_0007_gy': 1.77,
    'test_mae_case_0021_gy': 1.11,
    'test_mae_mean_gy': 1.44,
    'test_mae_std_gy': 0.33,
    'gamma_case_0007_pct': 41.2,
    'gamma_case_0021_pct': 14.5,
    'gamma_mean_pct': 27.9,
    'training_time_hours': 1.85,
    'early_stopped_epoch': 62,
}

print('Quantitative Results:')
for k, v in RESULTS.items():
    print(f'  {k}: {v}')

### 6.6 Per-Case Test Results

| Case | Test MAE (Gy) | Gamma (3%/3mm) |
|------|---------------|----------------|
| case_0007 | 1.77 | **41.2%** |
| case_0021 | 1.11 | 14.5% |
| **Mean** | **1.44 ± 0.33** | **27.9%** |

---

## 7. Analysis

### 7.1 Observations

1. **Gamma improvement is case-dependent:** case_0007 showed dramatic improvement (41.2% vs baseline ~9.5%), while case_0021 was similar to baseline (14.5% vs ~18.9%).

2. **MAE remains stable:** Test MAE is essentially unchanged (1.44 vs 1.43 Gy), confirming that gradient loss doesn't sacrifice dose accuracy.

3. **Training dynamics similar:** Best epoch at 12, same as baseline, suggesting gradient loss doesn't affect convergence speed.

4. **Gradient loss contribution:** The gradient loss term (~0.003-0.004) is about 30-40% of MSE loss magnitude, indicating it provides meaningful gradient signal.

### 7.2 Comparison to Baseline

| Metric | Baseline | Gradient Loss | Improvement |
|--------|----------|---------------|-------------|
| Val MAE | 3.73 Gy | 3.67 Gy | -1.6% |
| Test MAE | 1.43 Gy | 1.44 Gy | +0.7% (negligible) |
| Gamma | 14.2% | 27.9% | **+96%** |
| Training time | 2.55h | 1.85h | -27% (faster) |
| Best epoch | 12 | 12 | Same |

### 7.3 Limitations

1. **Small test set:** Only 2 test cases; results may not generalize. Need validation with larger dataset (n≥50).

2. **2D Gamma computation:** Gamma was computed on central slice only, not full 3D volume.

3. **Case variability:** Large difference between test cases (41.2% vs 14.5%) suggests anatomy-dependent effects.

4. **Still below target:** 27.9% Gamma is improved but still below the 50%+ interim target and 95% clinical target.

---

## 8. Conclusions

### Key Takeaways

1. **Gradient loss works:** The 3D Sobel gradient loss successfully improves Gamma pass rate by nearly 2x (14.2% → 27.9%).

2. **No accuracy tradeoff:** MAE remains equivalent, confirming we can improve edge preservation without sacrificing dose accuracy.

3. **Architecture validated:** This confirms that edge/gradient preservation is important for clinical dose prediction.

4. **Room for improvement:** Gamma is still below clinical targets, suggesting further improvements are needed (VGG loss, more data, architecture changes).

### Recommendation

**Proceed with Phase B:** Run gradient + VGG combined loss experiment to see if perceptual features can push Gamma even higher.

---

## 9. Next Steps

- [x] Complete gradient loss experiment (this notebook)
- [ ] **Phase B:** Run gradient + VGG combined loss (`grad_vgg_combined`)
- [ ] Gradient weight sweep (0.05, 0.1, 0.2) if Phase B shows promise
- [ ] Full 3D Gamma computation (not just central slice)
- [ ] Re-evaluate with 100+ cases when available
- [ ] Consider adversarial loss for sharper edges

---

## 10. Artifacts

| Artifact | Path |
|----------|------|
| Best Checkpoint | `runs/grad_loss_0.1/checkpoints/best-epoch=012-val/mae_gy=3.670.ckpt` |
| Metrics CSV | `runs/grad_loss_0.1/version_1/metrics.csv` |
| Training Config | `runs/grad_loss_0.1/training_config.json` |
| Test Cases | `runs/grad_loss_0.1/test_cases.json` |
| Predictions | `predictions/grad_loss_0.1_test/` |
| Figures (PNG) | `runs/grad_loss_0.1/figures/*.png` |
| Figures (PDF) | `runs/grad_loss_0.1/figures/*.pdf` |
| Figure Script | `scripts/generate_grad_loss_figures.py` |

---

*Notebook created: 2026-01-20*  
*Last updated: 2026-01-20*  
*Git commit at experiment start: `5d111a0`*  
*Git commit with results: `69d8e52`*